<a href="https://colab.research.google.com/github/pattrickx/wordle_breaker/blob/main/Termo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
!pip3 install unidecode

In [152]:
import pandas as pd
import unidecode 
df = pd.read_csv("/content/drive/MyDrive/My_projects/teste/palavras.txt")

In [153]:
df

,palavras
0,ababa
1,ababá
2,abacá
3,abaci
4,ábaco
...,...
3965,zuate
3966,zumbi
3967,zunir
3968,zurro


In [154]:
df["palavras"] = df["palavras"].apply(lambda r:unidecode.unidecode(r))
df = df[df["palavras"].apply(lambda r:len(r)==5)]
df.drop_duplicates(inplace=True)
df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,palavras
0,ababa
2,abaca
3,abaci
4,abaco
5,abada
...,...
3965,zuate
3966,zumbi
3967,zunir
3968,zurro


In [155]:
letras_certas = [False, False, False, False, False]
letras_erradas = []
letras_existentes = []# [{"letra":"e","posicao":[True, True, True, True, True]}]

In [156]:
def finde_letra(palavra,letras):
    for l in letras:
        if l in palavra:
            return True
    return False

def finde_letras_existentes(p,letras):
    cont = 0
    for l in letras:
        for idx in range(5):
            if l["posicao"][idx]:
                if l["letra"]==p[idx]:
                    cont+=1
                    break
            else:
                if l["letra"]==p[idx]:
                    return False
               
    if cont == len(letras):
        return True
    return False

def find_letras_certas(p,letras_certas):
    for i,letra in enumerate(letras_certas):
        if letra != False:
            if not letra==p[i]:
                return False
    return True


def get_letras_peso(letras_peso,palavras_com_letras_certas):
    for n,row in palavras_com_letras_certas.iterrows():
        for l in row["palavras"]:
            letras_peso[l]+=1
def get_word_weights(letras_peso,word):
    weights = 0
    letras = []
    for l in word:
        if not l in letras:
            letras.append(l)
            weights+=letras_peso[l]
    return weights

In [162]:
letras_certas = [False, False, "i", False, "o"]
letras_erradas = ["e","c","a","r","s","u","n","m","d"]
letras_existentes = [{"letra":"o","posicao":[False, False, False, False, True]}]

In [163]:
filto_erradas = df.apply(lambda row: not finde_letra(row["palavras"],letras_erradas), axis=1)
palavras_sem_letras_erradas = df[filto_erradas]

filtro_letras_existentes = palavras_sem_letras_erradas.apply(lambda row: finde_letras_existentes(row["palavras"],letras_existentes), axis=1)
palavras_com_letras_existentes = palavras_sem_letras_erradas[filtro_letras_existentes]

filtro_letras_certas = palavras_com_letras_existentes.apply(lambda row: find_letras_certas(row["palavras"],letras_certas), axis=1)
palavras_com_letras_certas = palavras_com_letras_existentes[filtro_letras_certas]

letras_peso={"a":0,"b":0,"c":0,"d":0,"e":0,"f":0,"g":0,"h":0,"i":0,
             "j":0,"k":0,"l":0,"m":0,"n":0,"o":0,"p":0,"q":0,"r":0,
             "s":0,"t":0,"u":0,"v":0,"w":0,"x":0,"y":0,"z":0}
get_letras_peso(letras_peso,palavras_com_letras_certas)

palavras_com_letras_certas["value"] = palavras_com_letras_certas.apply(lambda row: get_word_weights(letras_peso,row['palavras']), axis=1)
palavras_com_letras_certas.sort_values(by='value',ascending=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,palavras,value
1620,glifo,5
